In [96]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import  RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, roc_auc_score, roc_curve, auc, classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import KFold, GridSearchCV, StratifiedKFold
from sklearn.base import clone as sk_clone
from sklearn.model_selection import train_test_split
from sklearn import metrics

from joblib import Parallel, delayed

from itertools import product

from scipy.io import arff
from scipy.stats import pearsonr, spearmanr

import matplotlib.pyplot as plt
import os
import pickle

from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import StratifiedKFold

from math import sqrt
from joblib import Parallel, delayed
from datetime import datetime
from tqdm.autonotebook import tqdm

from sklearn import (pipeline, preprocessing)

In [33]:
short_names = {
    'Age': 'Age',
    'Height': 'Ht',
    'Weight': 'Wt',
    'Body Mass Index (BMI)': 'BMI',
    'Total Body Water (TBW)': 'TBW',
    'Extracellular Water (ECW)': 'ECW',
    'Intracellular Water (ICW)': 'ICW',
    'Extracellular Fluid/Total Body Water (ECF/TBW)': 'ECF/TBW',
    'Total Body Fat Ratio (TBFR) (%)': 'TBFR',
    'Lean Mass (LM) (%)': 'LM',
    'Body Protein Content (Protein) (%)': 'Protein',
    'Visceral Fat Rating (VFR)': 'VFR',
    'Bone Mass (BM)': 'BM',
    'Muscle Mass (MM)': 'MM',
    'Obesity (%)': 'Obesity',
    'Total Fat Content (TFC)': 'TFC',
    'Visceral Fat Area (VFA)': 'VFA',
    'Visceral Muscle Area (VMA) (Kg)': 'VMA',
    'Hepatic Fat Accumulation (HFA)': 'HFA',
    'Glucose': 'Glu',
    'Total Cholesterol (TC)': 'TC',
    'Low Density Lipoprotein (LDL)': 'LDL',
    'High Density Lipoprotein (HDL)': 'HDL',
    'Triglyceride': 'TG',
    'Aspartat Aminotransferaz (AST)': 'AST',
    'Alanin Aminotransferaz (ALT)': 'ALT',
    'Alkaline Phosphatase (ALP)': 'ALP',
    'Creatinine': 'Cr',
    'Glomerular Filtration Rate (GFR)': 'GFR',
    'C-Reactive Protein (CRP)': 'CRP',
    'Hemoglobin (HGB)': 'HGB',
    'Vitamin D': 'VitD'
}


In [3]:
def nested_cv (
    X, y, clf, param_grid, outer_folds=5, inner_folds=2, group_ids = False, shuffle=True,
    njobs=-1, print_params=False, scoring='roc_auc_ovr_weighted', verbose=0, debug=False, progress_bar=True
    ):
    '''
    nested cross-validation (nfolds x 2)
    outer validation holds out test fold and 
    inner validation gets the best estimator using grid search
    inner validation uses the outer cv's train fold (50-50 split)

    params:
    X = (standardized) features
    y = (prepocessed) label vector
    clf = initialized classifier
    param_grid = grid search parameter list
    outerFolds = outer fold count
    innerFolds = innerfold count
    group_ids = column contain group id of each row; folds keep groups intact
    njobs = how many cores to use (-1 = use all cores)
    scoring = scoring metric (string)
    verbose = pass verbose param to inner functions
    debug = print user-set debug messages
    '''
    n_classes = len(np.unique(y))
    
    # print(type(clf))
    # print (n_classes)
    
    testPredict = []
    Stest = []

    shuffle = False if group_ids is not False else shuffle

    KfoldStrategy = StratifiedKFold if group_ids is False else StratifiedGroupKFold
        
    outer_cv = KfoldStrategy(n_splits = outer_folds, random_state = 42 if shuffle else None, shuffle = shuffle)

    outer_splits = outer_cv.split(X, y) if group_ids is False else outer_cv.split(X, y, group_ids)

    
    fold_auc = []
    scores = []

    best_params_list = []
    fold_test_truth = []
    feature_importances = []
    # print('Start time', datetime.now().time())
    
#     kfold_progress = tqdm(outer_splits, total=outer_folds) if progress_bar else outer_splits
#     kfold_progress = progress_bar

    for k, (grid_ids, test_ids) in tqdm(enumerate(outer_splits)):
        
#         progress_bar and kfold_progress.set_description(f'Test Fold: {k+1}/{outer_folds}')
        
        # print(grid_ids.shape, test_ids.shape)
        # Accumulating training fold data and labels
        X_grid = X[grid_ids]
        y_grid  = y[grid_ids]
        
        # Accumulating test fold data and labels
        X_test = X[test_ids]
        y_test = y[test_ids]
        
#         progress_bar and kfold_progress.set_description(f'Doing GridSearch')

        inner_cv = KfoldStrategy(n_splits = inner_folds, random_state = 42 if shuffle else None, shuffle = shuffle)
        
        grid_search = GridSearchCV(estimator=clf, 
                                  param_grid = param_grid, n_jobs=njobs,
                                  scoring = scoring, cv = inner_cv, verbose=verbose)
        
    
        # Training and validation
#         progress_bar and kfold_progress.set_description(f'Fitting GridSearch')
        if group_ids is False:
            grid_search.fit(X_grid, y_grid)
        else:
            groups_grid = group_ids[grid_ids]
            grid_search.fit(X_grid, y_grid, groups = groups_grid)

#         progress_bar and kfold_progress.set_description(f'Fitted GridSearch')
        
        if print_params: print(grid_search.best_params_)
        
        best_params_list.append(grid_search.best_params_)
        
#         progress_bar and kfold_progress.set_description(f'Testing best estimator')
        best_model = grid_search.best_estimator_
        
        # test with the best model found in the validation step
        # print(X_test.shape, y_test.shape)
        y_hat =  best_model.predict(X_test)
        # print(X_test.shape, y_test.shape, y_hat.shape)
        
#         progress_bar and kfold_progress.clear_prefix()
        
        scores.append(best_model.score(X_test, y_test))
        
        fold_test_truth.append({
            'predicted':y_hat, 
            'actual': y_test})
        

        if isinstance(clf, LogisticRegression):
            feature_importance = abs(best_model.coef_[0])
            feature_importances.append(feature_importance)
            
        else:
            feature_importance = best_model.feature_importances_
            feature_importances.append(feature_importance)
    
    return scores, fold_test_truth, best_params_list, feature_importances

In [4]:
common_classifiers = {
    'Logistic Regression': {
        'model': LogisticRegression (random_state = 42, class_weight = "balanced", max_iter=300, solver='liblinear'),
        'params': {
            'C': [0.8, 0.5, 1, 5, 0.01,0.05], 
            'penalty': ['l1', 'l2']
        },
    },
    'Random Forest': {
        'model': RandomForestClassifier(n_estimators= 5,  class_weight = "balanced",random_state=42),
        'params': {
            'n_estimators': list(range(10, 120, 20))
            },
    },
    'Decision Tree': {
        'model': DecisionTreeClassifier(class_weight='balanced', random_state=42),
        'params': {
              "min_samples_split": [2, 10, 20],
              "max_depth": [2, 5, 10]
            },
    },
    'Gradient Boosting': {
        'model': GradientBoostingClassifier(n_estimators=100, random_state=42),
        'params': {
            "n_estimators":[50, 80, 110],
            # "min_samples_split": [2, 5, 10, 15, 20],
            # "learning_rate":[0.1, 0.3, 1],
            "max_depth": [2, 5, 10, 15]
        },
    },
    'SVM Linear': {
        'model': SVC(kernel='linear', random_state=42, class_weight = "balanced", probability= True),
        'params': {
            'C': [0.8, 0.5, 0.1, 0.05, 0.01]
        },
    },
    'SVM RBF': {
        'model': SVC(kernel='rbf', random_state=42,cache_size=20000, class_weight = "balanced", probability= True),
        'params': {
            'C': [0.8, 0.5, 0.1, 0.05, 0.01], 
            'gamma': [0.1, 0.02, 0.3, 0.5, 0.05, 0.01]
            },
    },

}

In [ ]:
### load data from the MLP result in the 

ds_id = 'gallstone_female' #'gallstone_male'
output_dir = f"result/{ds_id}"
with open(f"{output_dir}/results_epochs1000_lr0.001_batch128_None_None_weight_6432168_0.9.pkl", "rb") as f1:
    results = pickle.load(f1)
data = results['updated_dataset']

In [ ]:
condition1 = (data['class'] == 1) & (data['mlp_acc'] < 0.5)
condition2 = (data['class'] == 0) & (data['mlp_acc'] > 0.5)

filtered_data = data[~(condition1 | condition2)]
filtered_data.to_csv(f'{directory}/remove_wrong.csv')

,Age,Height,Weight,Body Mass Index (BMI),Total Body Water (TBW),Extracellular Water (ECW),Intracellular Water (ICW),Extracellular Fluid/Total Body Water (ECF/TBW),Total Body Fat Ratio (TBFR) (%),Lean Mass (LM) (%),...,High Density Lipoprotein (HDL),Triglyceride,Aspartat Aminotransferaz (AST),Alanin Aminotransferaz (ALT),Alkaline Phosphatase (ALP),Creatinine,Glomerular Filtration Rate (GFR),C-Reactive Protein (CRP),Hemoglobin (HGB),Vitamin D
0,0.203623,0.463434,0.013030,-0.177270,0.456202,0.699285,0.143272,0.427253,-0.197230,0.224823,...,-0.678820,-0.162090,-0.279339,-0.279526,-0.388240,-1.372057,0.564248,-0.166437,0.674406,1.071487
1,1.230679,1.044272,1.009852,0.477763,1.081179,1.725663,0.296312,1.195360,0.735436,-0.720671,...,-0.057996,0.601097,0.389652,-0.070576,-0.493211,1.294059,-1.249532,-0.375703,-0.041463,0.882816
2,0.483730,1.189482,-0.201970,-0.640587,-0.240887,0.297659,-0.583669,0.939325,0.109400,-0.093372,...,0.174813,-0.382566,-0.279339,0.138374,-0.283270,-0.038999,0.103390,-0.410853,0.674406,0.984054
4,0.763836,-0.262613,1.687429,1.755877,0.360052,1.770288,-0.889750,2.475540,2.115270,-2.122029,...,-0.678820,-0.450405,-0.279339,-0.488477,0.801421,-0.610310,0.129337,-0.223658,1.310733,-0.041213
5,-0.356589,0.753853,-0.104243,-0.400940,0.912916,0.743910,0.870213,-0.084819,-0.848818,0.887188,...,0.252416,-0.111211,-1.115578,-1.219802,-0.738141,-0.610310,0.631586,-0.403496,-0.280086,0.747831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,2.257736,-0.698241,0.117273,0.397881,-0.337037,-0.684094,0.105012,-0.753072,0.753323,-0.760933,...,0.174813,-0.229928,0.055156,0.138374,0.836411,4.150613,-3.086787,-0.236737,0.435783,-0.736996
101,0.577098,0.027806,0.508183,0.461787,0.191789,-1.130345,1.252813,-2.110062,-0.014530,0.036503,...,-0.174400,0.838533,-1.282826,-0.697427,-0.493211,-0.705528,0.236829,-0.446004,0.753947,-0.902659
102,-0.076483,-0.262613,-0.716669,-0.608634,-0.361075,-0.148592,-0.392369,0.171217,-0.772161,0.806666,...,-0.407209,-0.687841,-0.279339,-0.175051,0.031640,-0.800747,0.953445,-0.421480,0.515324,-0.414874
103,0.390361,-1.424289,0.006515,0.621551,-0.192812,0.208409,-0.277589,0.591116,0.677943,-0.667422,...,0.019607,-0.670881,1.225890,-0.070576,-0.283270,0.722749,-0.436543,-0.446004,-1.314118,-0.746200


In [ ]:
y = filtered_data['class']
X = filtered_data.drop(columns = ['class', 'mlp_acc'])

X = X.fillna(X.median())
mean = X.mean(axis=0, skipna=True)
std = X.std(axis=0, skipna=True)
X_imp = (X - mean)/std
X_imp

In [83]:
directory = f'result/{ds_id}'
if not os.path.exists(directory):
    os.makedirs(directory, exist_ok=True)
    
filename_result = f'{directory}/ml_result.pkl'

if os.path.exists(filename_result):
    with open(filename_result, 'rb') as f:
        try:
            metrics_dict_result = pickle.load(f)
        except EOFError:  # In case the file is empty or corrupted
            metrics_dict_result = {'Random Forest':{}, 'Gradient Boosting':{}, 'Logistic Regression':{}}
else:
    metrics_dict_result = {'Random Forest':{}, 'Gradient Boosting':{}, 'Logistic Regression':{}}

In [22]:
clf_list = ['Random Forest', 'Gradient Boosting', 'Logistic Regression']

clf_output = {}
for i in clf_list:
    clf = common_classifiers[i]['model']
    param_grid = common_classifiers[i]['params']


    output = nested_cv (X_imp.values, y.values, clf, param_grid, outer_folds=10, inner_folds=2)
    clf_output.update({i:output})
    
    a,b,c,d = output
    result_imp = {'acc_scores':a, 'fold_test_truth':b, 'best_params_list':c, 'feature_importances':d}
    metrics_dict_result[i] = result_imp
    with open(filename_result, 'wb') as f:
        pickle.dump(metrics_dict_result, f)
    
    print('all:',i)

0it [00:00, ?it/s]

all: Random Forest


0it [00:00, ?it/s]

all: Gradient Boosting


0it [00:00, ?it/s]

all: Logistic Regression


In [86]:
def feature_importance(directory):
    print(directory)
    with open(f"{directory}/ml_result.pkl", 'rb') as f:
        data_dict = pickle.load(f)
    f = pd.DataFrame(data_dict['Gradient Boosting']['feature_importances']).T
    f_lg = pd.DataFrame(data_dict['Logistic Regression']['feature_importances']).T

    data_path = f'{directory}/remove_wrong.csv'
    data = pd.read_csv(data_path)
    correlation_with_HF = data.drop(columns = ['class']).rename(columns = short_names).corr(method='pearson')['mlp_acc'].round(2).sort_values(ascending=False)

    data = data.drop(columns=['Unnamed: 0', 'mlp_acc','class'], errors='ignore')
    f.index = data.columns
    f_lg.index = data.columns
    mean_imp_score = f.mean(axis=1).rename(index = short_names)#.rename(index = smoke_change)
    mean_imp_score_lg = f_lg.mean(axis=1).rename(index = short_names)
#     std_imp_score = f.std(axis=1).rename(index = short_names)#.rename(index = smoke_change)
    mean_imp = f.mean(axis=1).rank(ascending=False).rename(index = short_names)
    mean_imp_lg = f_lg.mean(axis=1).rank(ascending=False).rename(index = short_names)
#     std_imp = f.std(axis=1)#.rank(ascending=False)
#     print(mean_imp.sort_values().head(10))
#     mean_imp = mean_imp.rename(index = short_names)#.rename(index = smoke_change)
#     imp = mean_imp.sort_values()
    return mean_imp_score, mean_imp_score_lg, correlation_with_HF#, prev

In [80]:
def bigger(weight_matrix):
    """
    Enforces one-directional causal relationships in the weight matrix
    while keeping the original weight values.

    Parameters:
    - weight_matrix (numpy.ndarray): The original weight matrix from NOTEARS (n x n).

    Returns:
    - numpy.ndarray: The processed weight matrix with one direction enforced.
    """
    # Ensure the matrix is square
    assert weight_matrix.shape[0] == weight_matrix.shape[1], "Weight matrix must be square."

    # Create a copy of the weight matrix
    one_direction_matrix = weight_matrix.copy()

    # Enforce one direction
    for i in range(weight_matrix.shape[0]):
        for j in range(i + 1, weight_matrix.shape[1]):  # Only process upper triangular part
            if weight_matrix[i, j] > weight_matrix[j, i]:
                one_direction_matrix[j, i] = 0  # Keep W[i, j] and remove W[j, i]
            else:
                one_direction_matrix[i, j] = 0  # Keep W[j, i] and remove W[i, j]

    return one_direction_matrix

In [70]:
imp_gbt, imp_lg, corr_f = feature_importance(directory)

In [105]:
def causality_weight(model, directory, ds_id):
    if model == 'LiNGAM':
        w = pd.read_csv(f'{directory}/LiNGAM_mlp_acc_{ds_id}_stand.csv')#class
#         w = pd.read_csv(f'results/causal/remove_out/{group}_LiNGAM_matrix.csv')_gbt
        w.index = w['Unnamed: 0']
        w = w.drop(columns = ['Unnamed: 0'])
        effect = w['mlp_acc'][w['mlp_acc'] != 0].rename(index = short_names)#.abs().rank(ascending=False)#.sort_values()
        cause = w.loc['mlp_acc'][w.loc['mlp_acc'] != 0].rename(index = short_names)#.abs().rank(ascending=False)#.sort_values() 
#         effect = w['mlp_label'][w['mlp_label'] != 0].abs().rename(index = short_names)#.rank(ascending=False)#.sort_values()
#         cause = w.loc['mlp_label'][w.loc['mlp_label'] != 0].abs().rename(index = short_names)#.rank(ascending=False)#.sort_values()        
#         effect = w['combine'][w['combine'] != 0].abs().rename(index = short_names)
#         cause = w.loc['combine'][w.loc['combine'] != 0].abs().rename(index = short_names)
#         effect = w['gbt_label'][w['gbt_label'] != 0].abs().rename(index = short_names)
#         cause = w.loc['gbt_label'][w.loc['gbt_label'] != 0].abs().rename(index = short_names)
    else:
        # dag = pd.read_csv(f'{directory}/{model}_mlp_acc_{ds_id}_stand.csv')#_gbt_gbt
        w = pd.read_csv(f'{directory}/{model}_mlp_acc_{ds_id}_stand_weight.csv')#class
        # dag.index = dag['Unnamed: 0']
        # dag = dag.drop(columns = ['Unnamed: 0'])
        w.index = w['Unnamed: 0']
        w = w.drop(columns = ['Unnamed: 0'])
        w1 = bigger(w.values)
        w1 = pd.DataFrame(w1, index= w.columns, columns = w.columns)
    #     print(dag*w).sort_values().rank(ascending=False)
#         cause = w[w>0.05]['combine'].drop(index = 'combine').rename(index = short_names)
#         effect = w[w>0.05].loc['combine'].drop(index = 'combine').rename(index = short_names)
#         cause = w[w>0.05]['gbt_label'].drop(index = 'gbt_label').rename(index = short_names)
#         effect = w[w>0.05].loc['gbt_label'].drop(index = 'gbt_label').rename(index = short_names)
#         cause = w[w>0.05]['mlp_label'].drop(index = 'mlp_label').rename(index = short_names)#.rank(ascending=False)#.sort_values()
#         effect = w[w>0.05].loc['mlp_label'].drop(index = 'mlp_label').rename(index = short_names)#.rank(ascending=False)#.sort_values()
        cause = w1[w1 > 0.01].loc['mlp_acc'].drop(index = 'mlp_acc').rename(index = short_names)#.rank(ascending=False)#.sort_values()
        effect = w1[w1 > 0.01]['mlp_acc'].drop(index = 'mlp_acc').rename(index = short_names)
        # cause = w1[w1 > 0.01].loc['class'].drop(index = 'class').rename(index = short_names)#.rank(ascending=False)#.sort_values()
        # effect = w1[w1 > 0.01]['class'].drop(index = 'class').rename(index = short_names)
    return cause, effect

In [108]:
groups = [ds_id]  # ['female', 'male_old', 'female_old', 'male_young', 'female_young']
corr_results = []
model = ['Notearsmlp', 'LiNGAM']
directory = f'result/{ds_id}'
for i in model:
    print('========================================', i, '==============================================')
    tables_cause = []
    tables_effect = []
    
    for group in groups:
        # Extract causality and feature importance
        print(directory)
        cause, effect = causality_weight(i, directory, ds_id)
        print(directory)
        imp_gbt, imp_lg, corr = feature_importance(directory)
        # corr = get_corr(group)
        
        # scaler = MinMaxScaler()
        
        # Table for Cause
        table_cause = pd.concat([cause, imp_gbt, imp_lg, corr], axis=1)#.rank(method='min', ascending=False)
        table_cause.columns = ['Cause', 'GBT_Imp', 'LG_Imp', 'F_Corr']
#         table_cause['LG_Imp'] = = table_cause['LG_Imp'] / table_cause['LG_Imp'].sum()
#         print(table_cause['LG_Imp'])
        table_cause = table_cause.dropna()
        table_cause['Cause_rank'] = table_cause['Cause'].abs().rank(ascending=False)
        table_cause['GBT_Imp_rank'] = table_cause['GBT_Imp'].rank(ascending=False)
        table_cause['LG_Imp_rank'] = table_cause['LG_Imp'].abs().rank(ascending=False)
        table_cause['F_Corr_rank'] = table_cause['F_Corr'].abs().rank(method='min', ascending=False)
#         table_cause['S_diff(C-GBT_Imp)'] = (table_cause['Cause'] - table_cause['GBT_Imp']) ** 2
#         table_cause['S_diff(C-LG_Imp)'] = (table_cause['Cause'] - table_cause['LG_Imp']) ** 2
        table_cause.columns = pd.MultiIndex.from_product([[group], table_cause.columns])
        tables_cause.append(table_cause)
        
        # Table for Effect
        table_effect = pd.concat([effect, imp_gbt, imp_lg, corr], axis=1)#.rank(method='min', ascending=False)
        table_effect.columns = ['Effect', 'GBT_Imp', 'LG_Imp', 'F_Corr']
#         table_effect['LG_Imp'] = table_effect['LG_Imp']
        table_effect = table_effect.dropna()
        table_effect['Effect_rank'] = table_effect['Effect'].abs().rank(ascending=False)
        table_effect['GBT_Imp_rank'] = table_effect['GBT_Imp'].rank(ascending=False)
        table_effect['LG_Imp_rank'] = table_effect['LG_Imp'].abs().rank(ascending=False)
        table_effect['F_Corr_rank'] = table_effect['F_Corr'].abs().rank(method='min', ascending=False)
#         table_effect['S_diff(E-GBT_Imp)'] = (table_effect['Effect'] - table_effect['GBT_Imp']) ** 2
#         table_effect['S_diff(E-LG_Imp)'] = (table_effect['Effect'] - table_effect['LG_Imp']) ** 2
        table_effect.columns = pd.MultiIndex.from_product([[group], table_effect.columns])
        tables_effect.append(table_effect)
    
    # Combine tables
    final_table_cause = pd.concat(tables_cause, axis=1)
    final_table_effect = pd.concat(tables_effect, axis=1)
    
    final_table_cause.to_csv(f'{directory}/{i}_cause_table.csv')
    final_table_effect.to_csv(f'{directory}/{i}_effect_table.csv')
    
    print("Table for Cause")
    print(final_table_cause.dropna())#.astype('int')
    print("\nTable for Effect")
    print(final_table_effect.dropna())#.astype('int')
    
    print("Table for Cause")
    # print(final_table_cause.dropna()['male'][['Cause_rank', 'GBT_Imp_rank', 'LG_Imp_rank', 'F_Corr_rank']].astype('int'))
    print("\nTable for Effect")
    # print(final_table_effect.dropna()['male'][['Effect_rank', 'GBT_Imp_rank', 'LG_Imp_rank', 'F_Corr_rank']].astype('int'))
    
#     print("\nLatex for Cause Table")
#     print(latex_table(final_table_cause.dropna().astype('int')))
#     print("\nLatex for Effect Table")
#     print(latex_table(final_table_effect.dropna().astype('int')))
    
    # Compute correlations for both tables
    relationships = [
#         ('Cause', 'F_Corr'), 
        ('Cause_rank', 'F_Corr_rank'),
#         ('F_Corr', 'GBT_Imp'), 
        ('Cause_rank', 'GBT_Imp_rank'),
        ('Cause_rank', 'LG_Imp_rank'),
#         ('F_Corr', 'LG_Imp'), 
#         ('Cause', 'LG_Imp'),
#         ('Effect', 'F_Corr'),
        ('Effect_rank', 'F_Corr_rank'),
        ('Effect_rank', 'GBT_Imp_rank'),
        ('Effect_rank', 'LG_Imp_rank'),
#         ('Effect', 'GBT_Imp'), ('Effect', 'LG_Imp')
    ]
    
    for group in groups:
        for y_col, x_col in relationships:
            
            if (group, x_col) in final_table_cause.columns and (group, y_col) in final_table_cause.columns:
                x = final_table_cause[(group, x_col)].astype(float)
                y = final_table_cause[(group, y_col)].astype(float)
                xy = pd.concat([x, y], axis=1).dropna()
#                 print(xy)
#                 xy['F_Corr_rank'] = xy[(group,'F_Corr')].abs().rank(method='min', ascending=False)
#                 xy['Cause_rank'] = y.abs().rank(ascending=False)
                print(xy)
                pearson_corr, pearson_p = pearsonr(xy[(group, y_col)], xy[(group, x_col)])
                spearman_corr, spearman_p = spearmanr(xy[(group, y_col)], xy[(group, x_col)], nan_policy='omit')
                corr_results.append([i, group, y_col, x_col, 'Cause', pearson_corr, pearson_p, spearman_corr, spearman_p])
                
#                 plt.figure(figsize=(8, 6))
#                 sns.regplot(x=x, y=y, ci=None, color = 'black', line_kws={"color": "black"})
                
# #                 print(x.index)
#                 for idx, txt in enumerate(x.index):
#                     plt.annotate(txt, (x.iloc[idx], y.iloc[idx] - 0.5), fontsize=12, color="black", alpha=0.7)
                
#                 # Set labels and title
#                 plt.xlabel(x_col, fontsize=13)
#                 plt.ylabel(y_col, fontsize=13)
#                 plt.xticks(fontsize=13)
#                 plt.yticks(fontsize=13)
# #                 plt.title(f'{group}: {x_col} vs {y_col}')
                
#                 # Display the correlation coefficient on the plot
#                 plt.text(0.05, 0.9, f'Spearman_corr: {spearman_corr:.2f}', transform=plt.gca().transAxes, fontsize=12, color="black")
#                 plt.tight_layout()
#                 plt.savefig(f'results/causal/figure_2/hf_icd_male_class_corr_{x_col} vs {y_col}_{i}_{group}_cause.pdf')
#                 plt.show()
                
#             if (group, x_col) in final_table_effect.columns and (group, y_col) in final_table_effect.columns:
#                 x = final_table_effect[(group, x_col)].astype(float)
#                 y = final_table_effect[(group, y_col)].astype(float)
#                 xy = pd.concat([x, y], axis=1).dropna()
# #                 xy['F_Corr_rank'] = xy[(group,'F_Corr')].abs().rank(method='min', ascending=False)
# #                 xy['Effect_rank'] = y.abs().rank(ascending=False)
# #                 print(xy)
#                 pearson_corr, pearson_p = pearsonr(xy[(group, y_col)], xy[(group, x_col)])
#                 spearman_corr, spearman_p = spearmanr(xy[(group, y_col)], xy[(group, x_col)], nan_policy='omit')
#                 corr_results.append([i, group, y_col, x_col, 'Effect', pearson_corr, pearson_p, spearman_corr, spearman_p])
                
# #                 plt.figure(figsize=(8, 6))
# #                 sns.regplot(x=x, y=y, ci=None,color = 'black', line_kws={"color": "black"})
                
# #                 # Add feature names beside the nodes
# #                 for idx, txt in enumerate(x.index):
# #                     plt.annotate(txt, (x.iloc[idx], y.iloc[idx] - 0.5), fontsize=12, color="black", alpha=0.7)
                
# #                 # Set labels and title
# #                 plt.xlabel(x_col, fontsize=13)
# #                 plt.ylabel(y_col, fontsize=13)
# #                 plt.xticks(fontsize=13)
# #                 plt.yticks(fontsize=13)
# # #                 plt.title(f'{group}: {x_col} vs {y_col}')
                
# #                 # Display the correlation coefficient on the plot
# #                 plt.text(0.05, 0.9, f'Spearman_corr: {spearman_corr:.2f}', transform=plt.gca().transAxes, fontsize=12, color="black")
# #                 plt.tight_layout()
# #                 plt.savefig(f'results/causal/figure_2/hf_icd_male_class_corr_{x_col} vs {y_col}_{i}_{group}_effect.pdf')
# #                 plt.show()

# Convert results to DataFrame
corr_df = pd.DataFrame(corr_results, columns=['Model', 'Group', 'Y', 'X', 'Table Type', 'Pearson Corr', 'Pearson p-value', 'Spearman Corr', 'Spearman p-value'])
corr_df.to_csv(f'{directory}/corrlation_table.csv')
corr_df

======================================== Notearsmlp ==============================================
result/gallstone_female
result/gallstone_female
result/gallstone_female
Table for Cause
     gallstone_female                                                     \
                Cause   GBT_Imp    LG_Imp F_Corr Cause_rank GBT_Imp_rank   
Glu          0.840728  0.099596  1.224716  -0.32        4.0          3.0   
LDL          0.285722  0.000642  0.109322  -0.05        7.0          9.0   
HDL          0.466718  0.009126  0.330912   0.14        5.0          5.0   
TG           0.133548  0.003225  0.083521   0.05        9.0          8.0   
AST          0.853421  0.008860  0.666199  -0.39        2.0          6.0   
ALT          0.247228  0.031441  0.212942  -0.04        8.0          4.0   
CRP          0.849873  0.313556  2.042605   0.37        3.0          2.0   
HGB          0.320761  0.005361  0.339577  -0.13        6.0          7.0   
VitD         1.140721  0.343696  1.601538  -0.56     

,Model,Group,Y,X,Table Type,Pearson Corr,Pearson p-value,Spearman Corr,Spearman p-value
0,Notearsmlp,gallstone_female,Cause_rank,F_Corr_rank,Cause,0.958396,0.000046,0.970720,0.000014
1,Notearsmlp,gallstone_female,Cause_rank,GBT_Imp_rank,Cause,0.666667,0.049867,0.666667,0.049867
2,Notearsmlp,gallstone_female,Cause_rank,LG_Imp_rank,Cause,0.883333,0.001591,0.883333,0.001591
3,LiNGAM,gallstone_female,Cause_rank,F_Corr_rank,Cause,0.463327,0.110809,0.445668,0.126938
4,LiNGAM,gallstone_female,Cause_rank,GBT_Imp_rank,Cause,0.214286,0.482054,0.214286,0.482054
5,LiNGAM,gallstone_female,Cause_rank,LG_Imp_rank,Cause,-0.153846,0.615799,-0.153846,0.615799
